In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
import calendar

MONTHS = calendar.month_name[1:]
MONTHS

['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [3]:
from datetime import date

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()
    
    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]
    
    x = [MONTHS[dt.month - 1] + ' ' + dt.strftime('%d, %Y') for dt in dates]
    y = [dt.isoformat() for dt in dates]
    
    return x, y

In [4]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [5]:
INPUT_CHARS = ''.join(sorted(set(''.join(MONTHS) + '0123456789, ')))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [6]:
OUTPUT_CHARS = '0123456789-'

In [7]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [8]:
print(x_example[0])
date_str_to_ids(x_example[0], INPUT_CHARS)

September 20, 7075


[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [9]:
print(y_example[0])
date_str_to_ids(y_example[0], OUTPUT_CHARS)

7075-09-20


[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [10]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    x_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    x = tf.ragged.constant(x_ids, ragged_rank=1)
    return (x + 1).to_tensor()

def create_dateset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [11]:
np.random.seed(42)

x_train, y_train = create_dateset(10000)
x_valid, y_valid = create_dateset(2000)
x_test, y_test = create_dateset(2000)

In [12]:
x_train[0]

<tf.Tensor: shape=(18,), dtype=int32, numpy=
array([20, 24, 32, 35, 24, 29, 22, 24, 33,  1,  5,  3,  2,  1, 10,  3, 10,
        8])>

In [13]:
y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1])>

basic seq2seq model

In [14]:
embedding_size = 32
max_output_length = y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, 
                           output_dim=embedding_size, 
                           input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation='softmax')
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(max_output_length),
    decoder
])

In [15]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['acc'])

history = model.fit(x_train, y_train, epochs=20, 
                    validation_data=(x_valid, y_valid))

Epoch 1/20
313/313 [==============================] - 8s 12ms/step - loss: 1.8040 - acc: 0.3534 - val_loss: 1.3620 - val_acc: 0.4954
Epoch 2/20
313/313 [==============================] - 3s 10ms/step - loss: 1.2083 - acc: 0.5599 - val_loss: 1.0384 - val_acc: 0.6218
Epoch 3/20
313/313 [==============================] - 3s 10ms/step - loss: 1.2340 - acc: 0.5710 - val_loss: 1.0927 - val_acc: 0.6130
Epoch 4/20
313/313 [==============================] - 3s 10ms/step - loss: 0.8616 - acc: 0.6829 - val_loss: 0.7503 - val_acc: 0.7151
Epoch 5/20
313/313 [==============================] - 3s 10ms/step - loss: 0.6409 - acc: 0.7538 - val_loss: 0.6277 - val_acc: 0.7466
Epoch 6/20
313/313 [==============================] - 3s 11ms/step - loss: 0.4652 - acc: 0.8169 - val_loss: 0.3821 - val_acc: 0.8492
Epoch 7/20
313/313 [==============================] - 3s 10ms/step - loss: 0.5064 - acc: 0.8149 - val_loss: 0.3457 - val_acc: 0.8672
Epoch 8/20
313/313 [==============================] - 3s 11ms/step - 

In [16]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return [''.join([('?' + chars)[index] for index in sequence]) 
            for sequence in ids]

In [17]:
x_new = prepare_date_strs(['September 17, 2009', 'July 14, 1789'])
ids = np.argmax(model.predict(x_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2009-09-17
1789-07-14


In [18]:
x_new = prepare_date_strs(['May 02, 2020', 'July 14, 1789'])
ids = np.argmax(model.predict(x_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2020-01-02
1789-01-14


In [19]:
max_input_length = x_train.shape[1]

def prepare_date_strs_padded(date_strs):
    x = prepare_date_strs(date_strs)
    if x.shape[1] < max_input_length:
        x = tf.pad(x, [[0, 0], [0, max_input_length - x.shape[1]]])
    return x

def convert_date_strs(date_strs):
    x = prepare_date_strs_padded(date_strs)
    ids = np.argmax(model.predict(x), axis=-1)
    return ids_to_date_strs(ids)

In [20]:
convert_date_strs(['May 02, 2020', 'July 14, 1789'])

['2020-05-02', '1789-07-14']

feeding the shifted targets to the decoder (teacher forcing)

In [21]:
sos_id = len(OUTPUT_CHARS) + 1
sos_id

12

In [22]:
def shifted_output_sequences(y):
    sos_tokens = tf.fill(dims=(len(y), 1), value=sos_id)
    return tf.concat([sos_tokens, y[:, :-1]], axis=1)

In [23]:
x_train_decoder = shifted_output_sequences(y_train)
x_valid_decoder = shifted_output_sequences(y_valid)
x_test_decoder = shifted_output_sequences(y_test)

In [24]:
x_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]])>

In [25]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(
    input_dim=len(INPUT_CHARS) + 1, 
    output_dim=encoder_embedding_size)(encoder_input)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
    lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2, 
    output_dim=decoder_embedding_size)(decoder_input)
decoder_lstm_output = keras.layers.LSTM(
    lstm_units, return_sequences=True)(decoder_embedding, 
                                       initial_state=encoder_state)
decoder_output = keras.layers.Dense(
    len(OUTPUT_CHARS) + 1, activation='softmax')(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], 
                           outputs=[decoder_output])

In [26]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['acc'])

history = model.fit([x_train, x_train_decoder], y_train, epochs=10, 
                    validation_data=([x_valid, x_valid_decoder], y_valid))

Epoch 1/10
313/313 [==============================] - 6s 12ms/step - loss: 1.6804 - acc: 0.3741 - val_loss: 1.4205 - val_acc: 0.4508
Epoch 2/10
313/313 [==============================] - 3s 11ms/step - loss: 1.1836 - acc: 0.5606 - val_loss: 0.8859 - val_acc: 0.6757
Epoch 3/10
313/313 [==============================] - 3s 11ms/step - loss: 0.6046 - acc: 0.7867 - val_loss: 0.3560 - val_acc: 0.8884
Epoch 4/10
313/313 [==============================] - 3s 11ms/step - loss: 0.1872 - acc: 0.9571 - val_loss: 0.1117 - val_acc: 0.9790
Epoch 5/10
313/313 [==============================] - 3s 11ms/step - loss: 0.0821 - acc: 0.9880 - val_loss: 0.0399 - val_acc: 0.9981
Epoch 6/10
313/313 [==============================] - 3s 11ms/step - loss: 0.0374 - acc: 0.9964 - val_loss: 0.0269 - val_acc: 0.9985
Epoch 7/10
313/313 [==============================] - 3s 11ms/step - loss: 0.0159 - acc: 0.9998 - val_loss: 0.0129 - val_acc: 0.9998
Epoch 8/10
313/313 [==============================] - 3s 11ms/step - 

In [27]:
sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    x = prepare_date_strs_padded(date_strs)
    y_pred = tf.fill(dims=(len(x), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - y_pred.shape[1]
        x_decoder = tf.pad(y_pred, [[0, 0], [0, pad_size]])
        y_probas_next = model.predict([x, x_decoder])[:, index:index+1]
        y_pred_next = tf.argmax(y_probas_next, axis=-1, output_type=tf.int32)
        y_pred = tf.concat([y_pred, y_pred_next], axis=1)
    return ids_to_date_strs(y_pred[:, 1:])

In [28]:
predict_date_strs(['May 02, 2020', 'July 14, 1789'])

['2020-05-02', '1789-07-14']